In [2]:
!pip install mlxtend

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# Ensures the output is displayed in a single line.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
# Step 1: Read the 'armut_data.csv' file.

In [5]:
df_ = pd.read_csv(r"C:\Users\User\Desktop\Miuul_Dersler\Case Studies\Armut Recommendation Systems\ArmutARL-221114-234936\armut_data.csv")
# Creating a copy as reading takes a long time.
df = df_.copy()
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
UserId,162523.0,13089.803862,7325.816060,0.0,6953.0,13139.0,19396.0,25744.0
ServiceId,162523.0,21.641140,13.774405,0.0,13.0,18.0,32.0,49.0
CategoryId,162523.0,4.325917,3.129292,0.0,1.0,4.0,6.0,11.0


In [7]:
df.isnull().sum()

UserId        0
ServiceId     0
CategoryId    0
CreateDate    0
dtype: int64

In [8]:
# Step 2: ServiceID represents different services for each CategoryID. 
# Create a new variable by combining ServiceID and CategoryID with an underscore '_'. 
# Expected output: UserId, ServiceId, CategoryId, CreateDate, Hizmet (Example: 4_5, 48_5, 0_8)

In [9]:
df['Hizmet'] = df['ServiceId'].astype(str) + "_" + df['CategoryId'].astype(str)

In [10]:
df.head(5)

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [11]:
# Step 3: The dataset consists of the dates and times services 
# were received without any basket definition (like invoice). 
# To apply Association Rule Learning, a basket definition must be created. 
# Here, the basket definition is the services each customer receives monthly. 
# For example, services 9_4 and 46_4 received by customer 7256 in August 2017 form one basket; 
# services 9_4 and 38_4 received in October 2017 form another basket. 
# Baskets need to be identified with a unique ID. 
# First, create a new date variable containing only year and month, 
# then combine UserID and this new date variable with an underscore '_' to create a new variable named ID.

In [12]:
# Convert the 'CreateDate' column to a datetime object
df['CreateDate'] = pd.to_datetime(df['CreateDate'])

In [13]:
# Create a new column and save the dates in 'Year-Month' format
df['New_Date'] = df['CreateDate'].dt.strftime('%Y-%m')

In [14]:
# Print the first few rows of the DataFrame to view the results
df.head(5)

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [15]:
df['SepetID'] = df['UserId'].astype(str) + "_" + df['New_Date']

In [16]:
df

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08
...,...,...,...,...,...,...,...
162518,10591,25,0,2018-08-06 14:40:00,25_0,2018-08,10591_2018-08
162519,10591,2,0,2018-08-06 14:43:00,2_0,2018-08,10591_2018-08
162520,10591,31,6,2018-08-06 14:47:00,31_6,2018-08,10591_2018-08
162521,12666,38,4,2018-08-06 16:01:00,38_4,2018-08,12666_2018-08


In [17]:
# Generating Association Rules
# Step 1: Create a hizmet, sepet pivot table

In [18]:
pivot_table = pd.pivot_table(df, index='SepetID', columns='Hizmet', aggfunc='size', fill_value=0)

In [19]:
pivot_table.head(5)

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [20]:
import numpy as np

In [21]:
unique_values = np.unique(pivot_table.values)

In [22]:
only_1_and_0 = np.all(np.isin(unique_values, [0, 1]))

In [23]:
print("Are there only 1 and 0 values in the pivot table?", only_1_and_0)
print("Unique values in the pivot table:", unique_values)

Are there only 1 and 0 values in the pivot table? False
Unique values in the pivot table: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 24 25 28
 29 41]


In [24]:
pivot_table_1 = pivot_table.applymap(lambda x: 1 if x != 0 else 0)

In [25]:
pivot_table_1.head(5)

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [26]:
unique_values_1 = np.unique(pivot_table_1.values)

In [27]:
only_1_and_0_changed = np.all(np.isin(unique_values_1, [0, 1]))

In [28]:
print("Are there only 1 and 0 values in the pivot table?", only_1_and_0_changed)
print("Unique values in the pivot table:", unique_values_1)

Are there only 1 and 0 values in the pivot table? True
Unique values in the pivot table: [0 1]


In [29]:
pivot_table_1 = pivot_table_1.astype(bool)

In [30]:
pivot_table_1.head(5)

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False
0_2017-09,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
0_2018-01,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
0_2018-04,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
10000_2017-08,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [31]:
# Apply the Apriori algorithm
frequent_itemsets = apriori(pivot_table_1, min_support=0.01, use_colnames=True)

In [32]:
frequent_itemsets.head(5)

,support,itemsets
0,0.019728,(0_8)
1,0.026523,(11_11)
2,0.029374,(12_7)
3,0.056627,(13_11)
4,0.023406,(14_7)


In [33]:
frequent_itemsets.sort_values("support", ascending=False).head(5)

,support,itemsets
8,0.238121,(18_4)
19,0.130286,(2_0)
5,0.120963,(15_1)
39,0.067762,(49_1)
28,0.066568,(38_4)


In [34]:
# Step 2: Generate association rules.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [35]:
# Interpreting the 'rules' variable:
# antecedents: Represents the first product. (x)
# consequents: Represents the second product. (y)
# antecedent support: Indicates the probability of observing the first product alone.
# consequent support: Indicates the probability of observing the second product alone.
# support: Indicates the probability of observing both products together.
# confidence: Indicates the probability of buying the second product (y) when the first product (x) is purchased.
# lift: Indicates how much the probability of buying the second product (y) increases when the first product (x) is purchased.

In [36]:
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047


In [37]:
rules[(rules["support"]>0.01) & (rules["confidence"]>0.05) & (rules["lift"]>3)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047
5,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728
10,(25_0),(22_0),0.042895,0.047515,0.011120,0.259247,5.456141,0.009082,1.285834,0.853324
11,(22_0),(25_0),0.047515,0.042895,0.011120,0.234043,5.456141,0.009082,1.249553,0.857462
18,(9_4),(38_4),0.041393,0.066568,0.010067,0.243216,3.653623,0.007312,1.233418,0.757661
19,(38_4),(9_4),0.066568,0.041393,0.010067,0.151234,3.653623,0.007312,1.129413,0.778096


In [38]:
# Step 3: Use the 'arl_recommender' function to make a service recommendation for a user who last received the 2_0 service.

In [39]:
# rec_count: Indicates the number of product recommendations to provide.
# sorted_rules: Sorts the rules in descending order based on lift.

def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for _, rule in sorted_rules.iterrows():
        if product_id in rule["antecedents"]:
            recommendation_list.extend(rule["consequents"])
    recommendation_list = list(set(recommendation_list))
    return recommendation_list[:rec_count]


In [40]:
arl_recommender(rules,"2_0",5)

['13_11', '38_4', '25_0', '22_0', '15_1']